# Ablation Study

In [ ]:
import os
import time
import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm_notebook as tqdm
import random
import glob

import cv2
from PIL import Image
from keras.layers import Input, Dense, BatchNormalization, Conv2D, Conv2DTranspose, ReLU, LeakyReLU, Flatten, MaxPooling2D, Dropout, Reshape

In [ ]:
# Define the path to the dataset
PATH1 = '/kaggle/input/dfuc-parta-resized/Abnormal'

# List all images in the folder
images = os.listdir(PATH1)
print(f'There are {len(images)} pictures of Abnormal.')

# Create a grid of 3x3 images
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 10))

for indx, axis in enumerate(axes.flatten()):
    # Randomly select an image
    rnd_indx = np.random.randint(0, len(images))
    img_path = os.path.join(PATH1, images[rnd_indx])  # Construct the correct file path
    img = plt.imread(img_path)  # Read the image
    imgplot = axis.imshow(img)  # Display the image
    axis.set_title(images[rnd_indx])  # Set the title to the image filename
    axis.set_axis_off()  # Turn off axis ticks and labels

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# Define the path to the dataset
PATH1 = '/kaggle/input/dfuc-parta-resized/Normal'

# List all images in the folder
images = os.listdir(PATH1)
print(f'There are {len(images)} pictures of Normal.')

# Create a grid of 3x3 images
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 10))

for indx, axis in enumerate(axes.flatten()):
    # Randomly select an image
    rnd_indx = np.random.randint(0, len(images))
    img_path = os.path.join(PATH1, images[rnd_indx])  # Construct the correct file path
    img = plt.imread(img_path)  # Read the image
    imgplot = axis.imshow(img)  # Display the image
    axis.set_title(images[rnd_indx])  # Set the title to the image filename
    axis.set_axis_off()  # Turn off axis ticks and labels

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Case 1:  Without Data Augmentation

In [ ]:
TRAIN_PATH = '/kaggle/input/dfuc-parta-resized'

BATCH_SIZE = 32
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
NUM_CLASSES = 2
EPOCHS = 200

In [ ]:
import os
import random
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image

# Path to your dataset (containing two subfolders: Normal/ and Abnormal/)
data_dir = "/kaggle/input/dfuc-parta-resized"  # Change to your dataset path

# Load dataset without any transform
train_data = ImageFolder(root=data_dir)

# Get class names (folder names)
class_names = train_data.classes

# Randomly select 10 image indices
indices = random.sample(range(len(train_data)), 10)

# Plot
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i, idx in enumerate(indices):
    path, label = train_data.samples[idx]  # This gives image path and label
    img = Image.open(path).convert("RGB")  # Open and ensure 3 channels

    axes[i].imshow(img)
    axes[i].set_title(f"Label: {class_names[label]}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Print class name details
print("Class Names:", class_names)
print("Index of 'Abnormal':", class_names.index("Abnormal"))
print("Index of 'Normal':", class_names.index("Normal"))


In [ ]:
def weights_init(m):
    """
    Takes as input a neural network m that will initialize all its weights.
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)  #Stabilizes early training, avoids large weights
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)   #Keeps initial scaling/shift minimal
        m.bias.data.fill_(0)

# Classification

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import callbacks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [ ]:
# Define Constants
TRAIN_PATH = '/kaggle/input/dfuc-parta-resized'

BATCH_SIZE = 32
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
NUM_CLASSES = 2
EPOCHS = 200

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/dfuc-parta-resized'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


In [ ]:
# Load the Training and Validation Dataset
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'training',
    seed = 0
)

validation_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'validation',
    seed = 0
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/dfuc-parta-resized"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


# Classification

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses
import numpy as np
from tensorflow.keras import metrics

# Self-Attention Layer
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim                      # Total embedding dimension
        self.num_heads = num_heads                      # Number of attention heads
        self.head_dim = embed_dim // num_heads          # Dimension per attention head
        
        # Ensure embedding dimension is divisible by number of heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
        
        # Dense layers to project inputs into queries, keys, and values
        self.query_dense = layers.Dense(embed_dim)  
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        
        # Dense layer to combine multiple attention heads output
        self.combine_heads = layers.Dense(embed_dim)
        
    def attention(self, query, key, value):
        # Calculate scaled dot-product attention scores
        score = tf.matmul(query, key, transpose_b=True)   # Compute dot product of query and key^T
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32) # Dimensionality for scaling
        scaled_score = score / tf.math.sqrt(dim_key)      # Scale scores to stabilize gradients
        weights = tf.nn.softmax(scaled_score, axis=-1)    # Apply softmax to get attention weights
        output = tf.matmul(weights, value)                 # Weighted sum of values based on attention weights
        return output, weights
    
    def separate_heads(self, x, batch_size):
        # Reshape the input tensor to split into multiple attention heads
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim)) 
        # Transpose to shape: (batch_size, num_heads, sequence_length, head_dim)
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        
        # Flatten spatial dimensions (height and width) into one sequence dimension
        input_shape = tf.shape(inputs)
        height, width = input_shape[1], input_shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, input_shape[3]))  # shape: (batch, seq_len, channels)
        
        # Create queries, keys, and values projections
        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)
        
        # Separate each into multiple heads
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        
        # Compute attention output and weights
        attention_output, attention_weights = self.attention(query, key, value)
        
        # Transpose back to (batch_size, seq_len, embed_dim)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        
        # Concatenate attention heads
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        
        # Final dense layer to combine heads into output
        output = self.combine_heads(concat_attention)
        
        # Reshape output back to spatial format (batch_size, height, width, embed_dim)
        output = tf.reshape(output, (batch_size, height, width, self.embed_dim))
        
        return output

# Spatial Attention Block
#SpatialAttentionBlock is designed to perform spatial attention—that is, it learns where in the image the model should "pay more attention" by assigning weights to each spatial location
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        # 1x1 convolutions for query, key, and value feature generation
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)  # Final convolution after attention
        self.softmax = layers.Softmax(axis=-1)    # Softmax for attention weights
        
    def call(self, inputs):
        # Get dynamic shapes for batch size, height, width, channels
        batch_size, height, width, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        
        # Generate query, key, and value feature maps from input
        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)
        
        # Reshape to (batch_size, sequence_length, filters) for matrix multiplication
        query = tf.reshape(query, (batch_size, height * width, self.filters))
        key = tf.reshape(key, (batch_size, height * width, self.filters))
        value = tf.reshape(value, (batch_size, height * width, self.filters))
        
        # Compute attention scores by dot product of query and key transpose
        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = attention_scores / tf.math.sqrt(tf.cast(self.filters, tf.float32))  # Scale scores
        
        # Softmax to obtain attention weights
        attention_weights = self.softmax(attention_scores)
        
        # Weighted sum of values based on attention weights
        attended_features = tf.matmul(attention_weights, value)
        
        # Reshape attended features back to spatial format
        attended_features = tf.reshape(attended_features, (batch_size, height, width, self.filters))
        
        # Final convolution to refine features
        output = self.conv_out(attended_features)
        
        # Add residual connection if input channels match filter count
        if inputs.shape[-1] == self.filters:
            output = output + inputs
        
        return output

# Channel Attention Block
#In Channel Attention, the model learns to weigh the importance of each channel in a feature map.
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio
        
        # Global average and max pooling layers
        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        
        # Dense layers to learn channel-wise attention weights
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')
        
    def call(self, inputs):
        # Average pooling branch: captures average channel information
        avg_pool = self.global_avg_pool(inputs)                     # Shape: (batch_size, channels)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)   # Reshape to (batch_size, 1, 1, channels)
        avg_pool = self.dense1(avg_pool)                             # Reduce dimension and non-linearity
        avg_pool = self.dense2(avg_pool)                             # Channel attention weights via sigmoid
        
        # Max pooling branch: captures prominent channel features
        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense1(max_pool)
        max_pool = self.dense2(max_pool)
        
        # Combine attention weights from both branches
        attention = avg_pool + max_pool
        
        # Multiply input features by attention weights (channel-wise scaling)
        return inputs * attention

# ResNet-like Block with Attention
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_attention=True, **kwargs):
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_attention = use_attention
        
        # Two convolutional layers with batch normalization
        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()
        
        # Attention blocks (spatial and channel) if enabled
        if use_attention:
            self.spatial_attention = SpatialAttentionBlock(filters)
            self.channel_attention = ChannelAttentionBlock(filters)
        
        self.relu = layers.ReLU()
        
    def call(self, inputs):
        # First conv layer with activation and batch norm
        x = self.conv1(inputs)
        x = self.bn1(x)
        
        # Second conv layer with batch norm (no activation yet)
        x = self.conv2(x)
        x = self.bn2(x)
        
        # Apply attention blocks if enabled
        if self.use_attention:
            x = self.spatial_attention(x)
            x = self.channel_attention(x)
        
        # Residual skip connection if input and output have same channel dimension
        if inputs.shape[-1] == self.filters:
            x = x + inputs
        
        # Final activation after residual addition
        return self.relu(x)

# CNN Model with Self-Attention
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    """
    Build CNN model with self-attention mechanisms
    
    Args:
        IMAGE_WIDTH: Input image width
        IMAGE_HEIGHT: Input image height  
        NUM_CLASSES: Number of output classes
    
    Returns:
        Compiled Keras model
    """
    
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))  # Input layer with image shape
    
    # Normalize pixel values to [0,1]
    x = layers.Rescaling(1./255)(inputs)
    
    # Initial convolutional layer with large kernel and stride for downsampling
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Stage 1: Basic convolutions with batch norm
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    # Stage 2: Residual blocks with attention + downsampling
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    
    # Stage 3: Increased filter count and residual attention blocks + downsampling
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    
    # Stage 4: High-level features + self-attention + downsampling
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    
    # Apply self-attention over feature map
    x = SelfAttention(embed_dim=512, num_heads=8)(x)
    
    # Additional residual attention blocks after self-attention
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    
    # Global average pooling to reduce spatial dimensions
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    
    # Dropout for regularization to prevent overfitting
    x = layers.Dropout(0.3)(x)
    
    # Dense layer with ReLU activation for learning complex features
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer:
    # For binary classification, use sigmoid activation and binary crossentropy loss
    # For multi-class, use softmax activation and sparse categorical crossentropy
    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()
    
    # Create the Keras model instance
    model = Model(inputs, outputs, name="CNN_SelfAttention")
    
    # Compile model with Adam optimizer, selected loss, and common classification metrics
    optimizer = optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=[
            'accuracy',                            # Classification accuracy
            metrics.Precision(name='precision'), # Precision metric
            metrics.Recall(name='recall'),       # Recall metric
            metrics.AUC(name='auc')               # Area under ROC curve
        ]
    )

    return model


# Usage example
if __name__ == "__main__":
    # Define input image dimensions and number of classes
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    NUM_CLASSES = 2
    
    print("Building CNN with Self-Attention model...")
    
    # Build the model
    model = build_cnn_attention_model(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES)
    
    # Print the model summary showing layers and parameters
    model.summary()
    
    # Generate a dummy input tensor (batch size 1, 224x224 RGB image)
    dummy_input = tf.random.normal((1, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    
    # Run the model on dummy data to test forward pass
    output = model(dummy_input)
    
    # Print output tensor shape
    print(f"\nModel output shape: {output.shape}")
    
    # Print actual output values (probabilities or logits)
    print(f"Output value: {output.numpy()}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [ ]:
inputs = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=200
)

# Case 2: Data Augmentation using CGAN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import callbacks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/resized-dataset-cgan/resized_dataset_CGAN'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


In [ ]:
# Load the Training and Validation Dataset
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'training',
    seed = 0
)

validation_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'validation',
    seed = 0
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/resized-dataset-cgan/resized_dataset_CGAN"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


# Classification

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses
import numpy as np
from tensorflow.keras import metrics

# Self-Attention Layer
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim                      # Total embedding dimension
        self.num_heads = num_heads                      # Number of attention heads
        self.head_dim = embed_dim // num_heads          # Dimension per attention head
        
        # Ensure embedding dimension is divisible by number of heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
        
        # Dense layers to project inputs into queries, keys, and values
        self.query_dense = layers.Dense(embed_dim)  
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        
        # Dense layer to combine multiple attention heads output
        self.combine_heads = layers.Dense(embed_dim)
        
    def attention(self, query, key, value):
        # Calculate scaled dot-product attention scores
        score = tf.matmul(query, key, transpose_b=True)   # Compute dot product of query and key^T
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32) # Dimensionality for scaling
        scaled_score = score / tf.math.sqrt(dim_key)      # Scale scores to stabilize gradients
        weights = tf.nn.softmax(scaled_score, axis=-1)    # Apply softmax to get attention weights
        output = tf.matmul(weights, value)                 # Weighted sum of values based on attention weights
        return output, weights
    
    def separate_heads(self, x, batch_size):
        # Reshape the input tensor to split into multiple attention heads
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim)) 
        # Transpose to shape: (batch_size, num_heads, sequence_length, head_dim)
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        
        # Flatten spatial dimensions (height and width) into one sequence dimension
        input_shape = tf.shape(inputs)
        height, width = input_shape[1], input_shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, input_shape[3]))  # shape: (batch, seq_len, channels)
        
        # Create queries, keys, and values projections
        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)
        
        # Separate each into multiple heads
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        
        # Compute attention output and weights
        attention_output, attention_weights = self.attention(query, key, value)
        
        # Transpose back to (batch_size, seq_len, embed_dim)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        
        # Concatenate attention heads
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        
        # Final dense layer to combine heads into output
        output = self.combine_heads(concat_attention)
        
        # Reshape output back to spatial format (batch_size, height, width, embed_dim)
        output = tf.reshape(output, (batch_size, height, width, self.embed_dim))
        
        return output

# Spatial Attention Block
#SpatialAttentionBlock is designed to perform spatial attention—that is, it learns where in the image the model should "pay more attention" by assigning weights to each spatial location
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        # 1x1 convolutions for query, key, and value feature generation
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)  # Final convolution after attention
        self.softmax = layers.Softmax(axis=-1)    # Softmax for attention weights
        
    def call(self, inputs):
        # Get dynamic shapes for batch size, height, width, channels
        batch_size, height, width, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        
        # Generate query, key, and value feature maps from input
        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)
        
        # Reshape to (batch_size, sequence_length, filters) for matrix multiplication
        query = tf.reshape(query, (batch_size, height * width, self.filters))
        key = tf.reshape(key, (batch_size, height * width, self.filters))
        value = tf.reshape(value, (batch_size, height * width, self.filters))
        
        # Compute attention scores by dot product of query and key transpose
        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = attention_scores / tf.math.sqrt(tf.cast(self.filters, tf.float32))  # Scale scores
        
        # Softmax to obtain attention weights
        attention_weights = self.softmax(attention_scores)
        
        # Weighted sum of values based on attention weights
        attended_features = tf.matmul(attention_weights, value)
        
        # Reshape attended features back to spatial format
        attended_features = tf.reshape(attended_features, (batch_size, height, width, self.filters))
        
        # Final convolution to refine features
        output = self.conv_out(attended_features)
        
        # Add residual connection if input channels match filter count
        if inputs.shape[-1] == self.filters:
            output = output + inputs
        
        return output

# Channel Attention Block
#In Channel Attention, the model learns to weigh the importance of each channel in a feature map.
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio
        
        # Global average and max pooling layers
        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        
        # Dense layers to learn channel-wise attention weights
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')
        
    def call(self, inputs):
        # Average pooling branch: captures average channel information
        avg_pool = self.global_avg_pool(inputs)                     # Shape: (batch_size, channels)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)   # Reshape to (batch_size, 1, 1, channels)
        avg_pool = self.dense1(avg_pool)                             # Reduce dimension and non-linearity
        avg_pool = self.dense2(avg_pool)                             # Channel attention weights via sigmoid
        
        # Max pooling branch: captures prominent channel features
        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense1(max_pool)
        max_pool = self.dense2(max_pool)
        
        # Combine attention weights from both branches
        attention = avg_pool + max_pool
        
        # Multiply input features by attention weights (channel-wise scaling)
        return inputs * attention

# ResNet-like Block with Attention
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_attention=True, **kwargs):
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_attention = use_attention
        
        # Two convolutional layers with batch normalization
        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()
        
        # Attention blocks (spatial and channel) if enabled
        if use_attention:
            self.spatial_attention = SpatialAttentionBlock(filters)
            self.channel_attention = ChannelAttentionBlock(filters)
        
        self.relu = layers.ReLU()
        
    def call(self, inputs):
        # First conv layer with activation and batch norm
        x = self.conv1(inputs)
        x = self.bn1(x)
        
        # Second conv layer with batch norm (no activation yet)
        x = self.conv2(x)
        x = self.bn2(x)
        
        # Apply attention blocks if enabled
        if self.use_attention:
            x = self.spatial_attention(x)
            x = self.channel_attention(x)
        
        # Residual skip connection if input and output have same channel dimension
        if inputs.shape[-1] == self.filters:
            x = x + inputs
        
        # Final activation after residual addition
        return self.relu(x)

# CNN Model with Self-Attention
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    """
    Build CNN model with self-attention mechanisms
    
    Args:
        IMAGE_WIDTH: Input image width
        IMAGE_HEIGHT: Input image height  
        NUM_CLASSES: Number of output classes
    
    Returns:
        Compiled Keras model
    """
    
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))  # Input layer with image shape
    
    # Normalize pixel values to [0,1]
    x = layers.Rescaling(1./255)(inputs)
    
    # Initial convolutional layer with large kernel and stride for downsampling
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Stage 1: Basic convolutions with batch norm
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    # Stage 2: Residual blocks with attention + downsampling
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    
    # Stage 3: Increased filter count and residual attention blocks + downsampling
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    
    # Stage 4: High-level features + self-attention + downsampling
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    
    # Apply self-attention over feature map
    x = SelfAttention(embed_dim=512, num_heads=8)(x)
    
    # Additional residual attention blocks after self-attention
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    
    # Global average pooling to reduce spatial dimensions
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    
    # Dropout for regularization to prevent overfitting
    x = layers.Dropout(0.3)(x)
    
    # Dense layer with ReLU activation for learning complex features
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer:
    # For binary classification, use sigmoid activation and binary crossentropy loss
    # For multi-class, use softmax activation and sparse categorical crossentropy
    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()
    
    # Create the Keras model instance
    model = Model(inputs, outputs, name="CNN_SelfAttention")
    
    # Compile model with Adam optimizer, selected loss, and common classification metrics
    optimizer = optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=[
            'accuracy',                            # Classification accuracy
            metrics.Precision(name='precision'), # Precision metric
            metrics.Recall(name='recall'),       # Recall metric
            metrics.AUC(name='auc')               # Area under ROC curve
        ]
    )

    return model


# Usage example
if __name__ == "__main__":
    # Define input image dimensions and number of classes
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    NUM_CLASSES = 2
    
    print("Building CNN with Self-Attention model...")
    
    # Build the model
    model = build_cnn_attention_model(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES)
    
    # Print the model summary showing layers and parameters
    model.summary()
    
    # Generate a dummy input tensor (batch size 1, 224x224 RGB image)
    dummy_input = tf.random.normal((1, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    
    # Run the model on dummy data to test forward pass
    output = model(dummy_input)
    
    # Print output tensor shape
    print(f"\nModel output shape: {output.shape}")
    
    # Print actual output values (probabilities or logits)
    print(f"Output value: {output.numpy()}")


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=200
)

# Case 3: Data Augmentation using Enhanced CGAN

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


# Load the Training and Validation Dataset
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'training',
    seed = 0
)

validation_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'validation',
    seed = 0
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


# Classification

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses
import numpy as np
from tensorflow.keras import metrics

# Self-Attention Layer
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim                      # Total embedding dimension
        self.num_heads = num_heads                      # Number of attention heads
        self.head_dim = embed_dim // num_heads          # Dimension per attention head
        
        # Ensure embedding dimension is divisible by number of heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
        
        # Dense layers to project inputs into queries, keys, and values
        self.query_dense = layers.Dense(embed_dim)  
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        
        # Dense layer to combine multiple attention heads output
        self.combine_heads = layers.Dense(embed_dim)
        
    def attention(self, query, key, value):
        # Calculate scaled dot-product attention scores
        score = tf.matmul(query, key, transpose_b=True)   # Compute dot product of query and key^T
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32) # Dimensionality for scaling
        scaled_score = score / tf.math.sqrt(dim_key)      # Scale scores to stabilize gradients
        weights = tf.nn.softmax(scaled_score, axis=-1)    # Apply softmax to get attention weights
        output = tf.matmul(weights, value)                 # Weighted sum of values based on attention weights
        return output, weights
    
    def separate_heads(self, x, batch_size):
        # Reshape the input tensor to split into multiple attention heads
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim)) 
        # Transpose to shape: (batch_size, num_heads, sequence_length, head_dim)
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        
        # Flatten spatial dimensions (height and width) into one sequence dimension
        input_shape = tf.shape(inputs)
        height, width = input_shape[1], input_shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, input_shape[3]))  # shape: (batch, seq_len, channels)
        
        # Create queries, keys, and values projections
        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)
        
        # Separate each into multiple heads
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        
        # Compute attention output and weights
        attention_output, attention_weights = self.attention(query, key, value)
        
        # Transpose back to (batch_size, seq_len, embed_dim)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        
        # Concatenate attention heads
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        
        # Final dense layer to combine heads into output
        output = self.combine_heads(concat_attention)
        
        # Reshape output back to spatial format (batch_size, height, width, embed_dim)
        output = tf.reshape(output, (batch_size, height, width, self.embed_dim))
        
        return output

# Spatial Attention Block
#SpatialAttentionBlock is designed to perform spatial attention—that is, it learns where in the image the model should "pay more attention" by assigning weights to each spatial location
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        # 1x1 convolutions for query, key, and value feature generation
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)  # Final convolution after attention
        self.softmax = layers.Softmax(axis=-1)    # Softmax for attention weights
        
    def call(self, inputs):
        # Get dynamic shapes for batch size, height, width, channels
        batch_size, height, width, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        
        # Generate query, key, and value feature maps from input
        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)
        
        # Reshape to (batch_size, sequence_length, filters) for matrix multiplication
        query = tf.reshape(query, (batch_size, height * width, self.filters))
        key = tf.reshape(key, (batch_size, height * width, self.filters))
        value = tf.reshape(value, (batch_size, height * width, self.filters))
        
        # Compute attention scores by dot product of query and key transpose
        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = attention_scores / tf.math.sqrt(tf.cast(self.filters, tf.float32))  # Scale scores
        
        # Softmax to obtain attention weights
        attention_weights = self.softmax(attention_scores)
        
        # Weighted sum of values based on attention weights
        attended_features = tf.matmul(attention_weights, value)
        
        # Reshape attended features back to spatial format
        attended_features = tf.reshape(attended_features, (batch_size, height, width, self.filters))
        
        # Final convolution to refine features
        output = self.conv_out(attended_features)
        
        # Add residual connection if input channels match filter count
        if inputs.shape[-1] == self.filters:
            output = output + inputs
        
        return output

# Channel Attention Block
#In Channel Attention, the model learns to weigh the importance of each channel in a feature map.
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio
        
        # Global average and max pooling layers
        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        
        # Dense layers to learn channel-wise attention weights
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')
        
    def call(self, inputs):
        # Average pooling branch: captures average channel information
        avg_pool = self.global_avg_pool(inputs)                     # Shape: (batch_size, channels)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)   # Reshape to (batch_size, 1, 1, channels)
        avg_pool = self.dense1(avg_pool)                             # Reduce dimension and non-linearity
        avg_pool = self.dense2(avg_pool)                             # Channel attention weights via sigmoid
        
        # Max pooling branch: captures prominent channel features
        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense1(max_pool)
        max_pool = self.dense2(max_pool)
        
        # Combine attention weights from both branches
        attention = avg_pool + max_pool
        
        # Multiply input features by attention weights (channel-wise scaling)
        return inputs * attention

# ResNet-like Block with Attention
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_attention=True, **kwargs):
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_attention = use_attention
        
        # Two convolutional layers with batch normalization
        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()
        
        # Attention blocks (spatial and channel) if enabled
        if use_attention:
            self.spatial_attention = SpatialAttentionBlock(filters)
            self.channel_attention = ChannelAttentionBlock(filters)
        
        self.relu = layers.ReLU()
        
    def call(self, inputs):
        # First conv layer with activation and batch norm
        x = self.conv1(inputs)
        x = self.bn1(x)
        
        # Second conv layer with batch norm (no activation yet)
        x = self.conv2(x)
        x = self.bn2(x)
        
        # Apply attention blocks if enabled
        if self.use_attention:
            x = self.spatial_attention(x)
            x = self.channel_attention(x)
        
        # Residual skip connection if input and output have same channel dimension
        if inputs.shape[-1] == self.filters:
            x = x + inputs
        
        # Final activation after residual addition
        return self.relu(x)

# CNN Model with Self-Attention
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    """
    Build CNN model with self-attention mechanisms
    
    Args:
        IMAGE_WIDTH: Input image width
        IMAGE_HEIGHT: Input image height  
        NUM_CLASSES: Number of output classes
    
    Returns:
        Compiled Keras model
    """
    
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))  # Input layer with image shape
    
    # Normalize pixel values to [0,1]
    x = layers.Rescaling(1./255)(inputs)
    
    # Initial convolutional layer with large kernel and stride for downsampling
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Stage 1: Basic convolutions with batch norm
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    # Stage 2: Residual blocks with attention + downsampling
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    
    # Stage 3: Increased filter count and residual attention blocks + downsampling
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    
    # Stage 4: High-level features + self-attention + downsampling
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    
    # Apply self-attention over feature map
    x = SelfAttention(embed_dim=512, num_heads=8)(x)
    
    # Additional residual attention blocks after self-attention
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    
    # Global average pooling to reduce spatial dimensions
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    
    # Dropout for regularization to prevent overfitting
    x = layers.Dropout(0.3)(x)
    
    # Dense layer with ReLU activation for learning complex features
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer:
    # For binary classification, use sigmoid activation and binary crossentropy loss
    # For multi-class, use softmax activation and sparse categorical crossentropy
    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()
    
    # Create the Keras model instance
    model = Model(inputs, outputs, name="CNN_SelfAttention")
    
    # Compile model with Adam optimizer, selected loss, and common classification metrics
    optimizer = optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=[
            'accuracy',                            # Classification accuracy
            metrics.Precision(name='precision'), # Precision metric
            metrics.Recall(name='recall'),       # Recall metric
            metrics.AUC(name='auc')               # Area under ROC curve
        ]
    )

    return model


# Usage example
if __name__ == "__main__":
    # Define input image dimensions and number of classes
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    NUM_CLASSES = 2
    
    print("Building CNN with Self-Attention model...")
    
    # Build the model
    model = build_cnn_attention_model(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES)
    
    # Print the model summary showing layers and parameters
    model.summary()
    
    # Generate a dummy input tensor (batch size 1, 224x224 RGB image)
    dummy_input = tf.random.normal((1, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    
    # Run the model on dummy data to test forward pass
    output = model(dummy_input)
    
    # Print output tensor shape
    print(f"\nModel output shape: {output.shape}")
    
    # Print actual output values (probabilities or logits)
    print(f"Output value: {output.numpy()}")


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=200
)

# Case 4: Data Augmentation using Enhanced CGAN,	Dropout: 0.3	No. of epochs: 200, No. of layers: 17	Self-attention: YES,	Spatial attention: NO

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


# Load the Training and Validation Dataset
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'training',
    seed = 0
)

validation_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'validation',
    seed = 0
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


# Classification

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses
from tensorflow.keras import metrics

# Self-Attention Layer
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        shape = tf.shape(inputs)
        height, width = shape[1], shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, shape[3]))
        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention_output, _ = self.attention(query, key, value)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        output = self.combine_heads(concat_attention)
        output = tf.reshape(output, (batch_size, height, width, self.embed_dim))
        return output

# Channel Attention Block
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio
        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')

    def call(self, inputs):
        avg_pool = self.global_avg_pool(inputs)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)
        avg_pool = self.dense1(avg_pool)
        avg_pool = self.dense2(avg_pool)

        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense1(max_pool)
        max_pool = self.dense2(max_pool)

        attention = avg_pool + max_pool
        return inputs * attention

# Residual Block with Only Channel Attention (Spatial attention disabled)
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_channel_attention=True, use_spatial_attention=False, **kwargs):  # --- MODIFIED for Ablation Case 4 ---
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_channel_attention = use_channel_attention
        self.use_spatial_attention = use_spatial_attention

        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()

        if self.use_channel_attention:
            self.channel_attention = ChannelAttentionBlock(filters)
        # SpatialAttentionBlock is disabled per ablation case

        self.relu = layers.ReLU()

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)

        if self.use_channel_attention:
            x = self.channel_attention(x)
        # Skipping spatial attention

        if inputs.shape[-1] == self.filters:
            x = x + inputs
        return self.relu(x)

# CNN Model with Self-Attention and Dropout
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Stage 1
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # Stage 2
    x = ResidualAttentionBlock(128, use_channel_attention=True, use_spatial_attention=False)(x)  # --- MODIFIED for Ablation Case 4 ---
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_channel_attention=True, use_spatial_attention=False)(x)  # --- MODIFIED

    # Stage 3
    x = ResidualAttentionBlock(256, use_channel_attention=True, use_spatial_attention=False)(x)  # --- MODIFIED
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_channel_attention=True, use_spatial_attention=False)(x)  # --- MODIFIED
    x = ResidualAttentionBlock(256, use_channel_attention=True, use_spatial_attention=False)(x)  # --- MODIFIED

    # Stage 4
    x = ResidualAttentionBlock(512, use_channel_attention=True, use_spatial_attention=False)(x)  # --- MODIFIED
    x = layers.MaxPooling2D(2)(x)

    # Self-Attention enabled
    x = SelfAttention(embed_dim=512, num_heads=8)(x)

    # Post Self-Attention Residuals
    x = ResidualAttentionBlock(512, use_channel_attention=True, use_spatial_attention=False)(x)  # --- MODIFIED
    x = ResidualAttentionBlock(512, use_channel_attention=True, use_spatial_attention=False)(x)  # --- MODIFIED

    # Global Pooling + Dropout + Dense
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)  # --- Ensures Dropout 0.3 as per Ablation Case 4 ---
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()

    model = Model(inputs, outputs, name="CNN_SelfAttention")
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss=loss_fn,
        metrics=[
            'accuracy',
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc')
        ]
    )
    return model

# Usage Example
if __name__ == "__main__":
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    NUM_CLASSES = 2

    print("Building CNN for Ablation Case 4 (Self-Attention + No Spatial Attention)...")
    model = build_cnn_attention_model(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES)
    model.summary()
    dummy_input = tf.random.normal((1, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    output = model(dummy_input)
    print(f"\nModel output shape: {output.shape}")
    print(f"Output value: {output.numpy()}")


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=200
)

# Case 5: Data Augmentation using Enhanced CGAN,	Dropout: 0.5	No. of epochs: 200, No. of layers: 17	Self-attention: NO,	Spatial attention: YES

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


# Load the Training and Validation Dataset
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'training',
    seed = 0
)

validation_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'validation',
    seed = 0
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


# Classification

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses
from tensorflow.keras import metrics

# Self-Attention Layer (NOT USED IN THIS CASE)
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        shape = tf.shape(inputs)
        height, width = shape[1], shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, shape[3]))
        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention_output, _ = self.attention(query, key, value)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        output = self.combine_heads(concat_attention)
        output = tf.reshape(output, (batch_size, height, width, self.embed_dim))
        return output

# Spatial Attention Block (ENABLED IN CASE 5)
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)
        self.softmax = layers.Softmax(axis=-1)

    def call(self, inputs):
        batch_size, height, width, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)

        query = tf.reshape(query, (batch_size, height * width, self.filters))
        key = tf.reshape(key, (batch_size, height * width, self.filters))
        value = tf.reshape(value, (batch_size, height * width, self.filters))

        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = attention_scores / tf.math.sqrt(tf.cast(self.filters, tf.float32))
        attention_weights = self.softmax(attention_scores)
        attended_features = tf.matmul(attention_weights, value)
        attended_features = tf.reshape(attended_features, (batch_size, height, width, self.filters))
        output = self.conv_out(attended_features)

        if inputs.shape[-1] == self.filters:
            output = output + inputs
        return output

# Channel Attention Block
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio
        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')

    def call(self, inputs):
        avg_pool = self.global_avg_pool(inputs)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)
        avg_pool = self.dense1(avg_pool)
        avg_pool = self.dense2(avg_pool)

        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense1(max_pool)
        max_pool = self.dense2(max_pool)

        attention = avg_pool + max_pool
        return inputs * attention

# Residual Block with Spatial Attention ENABLED and Self-Attention DISABLED
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_channel_attention=True, use_spatial_attention=True, **kwargs):  # --- MODIFIED for CASE 5 ---
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_channel_attention = use_channel_attention
        self.use_spatial_attention = use_spatial_attention

        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()

        if self.use_channel_attention:
            self.channel_attention = ChannelAttentionBlock(filters)
        if self.use_spatial_attention:  # --- ENABLED ---
            self.spatial_attention = SpatialAttentionBlock(filters)

        self.relu = layers.ReLU()

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)

        if self.use_spatial_attention:   # --- ENABLED in Case 5 ---
            x = self.spatial_attention(x)
        if self.use_channel_attention:
            x = self.channel_attention(x)

        if inputs.shape[-1] == self.filters:
            x = x + inputs
        return self.relu(x)

# CNN Model Builder
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Stage 1
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # Stage 2
    x = ResidualAttentionBlock(128, use_channel_attention=True, use_spatial_attention=True)(x)  # --- MODIFIED
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_channel_attention=True, use_spatial_attention=True)(x)

    # Stage 3
    x = ResidualAttentionBlock(256, use_channel_attention=True, use_spatial_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_channel_attention=True, use_spatial_attention=True)(x)
    x = ResidualAttentionBlock(256, use_channel_attention=True, use_spatial_attention=True)(x)

    # Stage 4
    x = ResidualAttentionBlock(512, use_channel_attention=True, use_spatial_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)

    #  Self-Attention is DISABLED in Case 5 — Skipped

    # Post Stage 4 Residual Blocks
    x = ResidualAttentionBlock(512, use_channel_attention=True, use_spatial_attention=True)(x)
    x = ResidualAttentionBlock(512, use_channel_attention=True, use_spatial_attention=True)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dropout(0.5)(x)  # --- MODIFIED: Dropout increased to 0.5
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)  # --- MODIFIED: Dropout increased to 0.5

    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()

    model = Model(inputs, outputs, name="CNN_SpatialAttention")
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss=loss_fn,
        metrics=[
            'accuracy',
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc')
        ]
    )
    return model

# Run the model
if __name__ == "__main__":
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    NUM_CLASSES = 2

    print("Building CNN for Ablation Case 5 (Spatial Attention = YES, Self-Attention = NO)...")
    model = build_cnn_attention_model(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES)
    model.summary()
    dummy_input = tf.random.normal((1, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    output = model(dummy_input)
    print(f"\nModel output shape: {output.shape}")
    print(f"Output value: {output.numpy()}")


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=200
)

# Case 6: Data Augmentation using Enhanced CGAN,	Dropout: 0.5	No. of epochs: 300, No. of layers: 16	Self-attention: YES,	Spatial attention: YES

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


# Load the Training and Validation Dataset
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'training',
    seed = 0
)

validation_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'validation',
    seed = 0
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


# Classification

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses, metrics

# Self-Attention Layer (ENABLED in Case 6)
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == embed_dim
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        shape = tf.shape(inputs)
        height, width = shape[1], shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, shape[3]))
        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention_output, _ = self.attention(query, key, value)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return tf.reshape(output, (batch_size, height, width, self.embed_dim))

# Spatial Attention Block (ENABLED)
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)
        self.softmax = layers.Softmax(axis=-1)

    def call(self, inputs):
        batch, h, w, _ = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)
        query = tf.reshape(query, (batch, h * w, self.filters))
        key = tf.reshape(key, (batch, h * w, self.filters))
        value = tf.reshape(value, (batch, h * w, self.filters))
        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores /= tf.math.sqrt(tf.cast(self.filters, tf.float32))
        attention_weights = self.softmax(attention_scores)
        attended_features = tf.matmul(attention_weights, value)
        attended_features = tf.reshape(attended_features, (batch, h, w, self.filters))
        output = self.conv_out(attended_features)
        if inputs.shape[-1] == self.filters:
            output += inputs
        return output

# Channel Attention Block
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.avg_pool = layers.GlobalAveragePooling2D()
        self.max_pool = layers.GlobalMaxPooling2D()
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')

    def call(self, inputs):
        avg = self.dense2(self.dense1(tf.expand_dims(tf.expand_dims(self.avg_pool(inputs), 1), 1)))
        max = self.dense2(self.dense1(tf.expand_dims(tf.expand_dims(self.max_pool(inputs), 1), 1)))
        return inputs * (avg + max)

# Residual Block with Attention
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_channel_attention=True, use_spatial_attention=True, **kwargs):
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_channel_attention = use_channel_attention
        self.use_spatial_attention = use_spatial_attention
        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()
        if use_spatial_attention:
            self.spatial_attention = SpatialAttentionBlock(filters)
        if use_channel_attention:
            self.channel_attention = ChannelAttentionBlock(filters)
        self.relu = layers.ReLU()

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.use_spatial_attention:
            x = self.spatial_attention(x)
        if self.use_channel_attention:
            x = self.channel_attention(x)
        if inputs.shape[-1] == self.filters:
            x += inputs
        return self.relu(x)

# Main Model Builder
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Stage 1
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # Stage 2 (2 residual blocks)
    x = ResidualAttentionBlock(128, use_channel_attention=True, use_spatial_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_channel_attention=True, use_spatial_attention=True)(x)

    # Stage 3 (2 residual blocks)
    x = ResidualAttentionBlock(256, use_channel_attention=True, use_spatial_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_channel_attention=True, use_spatial_attention=True)(x)

    # Stage 4 (2 residual blocks)
    x = ResidualAttentionBlock(512, use_channel_attention=True, use_spatial_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)

    # Self-Attention enabled in Case 6
    x = SelfAttention(embed_dim=512)(x)

    # One fewer residual block to match 16 total layers
    x = ResidualAttentionBlock(512, use_channel_attention=True, use_spatial_attention=True)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)

    #  Dropout rate = 0.5
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()

    model = Model(inputs, outputs, name="CNN_Attn_Case6")
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss=loss_fn,
        metrics=['accuracy', metrics.Precision(), metrics.Recall(), metrics.AUC()]
    )
    return model

# Run model for test
if __name__ == "__main__":
    model = build_cnn_attention_model(224, 224, 2)
    model.summary()
    dummy_input = tf.random.normal((1, 224, 224, 3))
    output = model(dummy_input)
    print(f"\nModel Output: {output.numpy()}")


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=300
)

# Case 7: Data Augmentation using Enhanced CGAN,	Dropout: 0.3,	No. of epochs: 200, No. of layers: 16	Self-attention: NO,	Spatial attention: NO

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


# Load the Training and Validation Dataset
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'training',
    seed = 0
)

validation_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'validation',
    seed = 0
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


# Classification

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses, metrics

# -----------------------------
# Channel Attention Block (Unchanged, but disabled in this config)
# -----------------------------
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio
        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')

    def call(self, inputs):
        avg_pool = self.global_avg_pool(inputs)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)
        avg_pool = self.dense2(self.dense1(avg_pool))

        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense2(self.dense1(max_pool))

        attention = avg_pool + max_pool
        return inputs * attention

# -----------------------------
# Spatial Attention Block (Unchanged, but disabled in this config)
# -----------------------------
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)
        self.softmax = layers.Softmax(axis=-1)

    def call(self, inputs):
        batch_size, h, w, c = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)

        query = tf.reshape(query, (batch_size, h * w, -1))
        key = tf.reshape(key, (batch_size, h * w, -1))
        value = tf.reshape(value, (batch_size, h * w, -1))

        score = tf.matmul(query, key, transpose_b=True)
        score = score / tf.math.sqrt(tf.cast(tf.shape(key)[-1], tf.float32))
        weights = self.softmax(score)

        attended = tf.matmul(weights, value)
        attended = tf.reshape(attended, (batch_size, h, w, -1))
        out = self.conv_out(attended)

        if inputs.shape[-1] == out.shape[-1]:
            out += inputs

        return out

# -----------------------------
# Residual Block with toggle switches for attention
# -----------------------------
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_spatial_attention=False, use_channel_attention=False, **kwargs):
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_spatial_attention = use_spatial_attention
        self.use_channel_attention = use_channel_attention

        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()

        if use_spatial_attention:
            self.spatial_attention = SpatialAttentionBlock(filters)
        if use_channel_attention:
            self.channel_attention = ChannelAttentionBlock(filters)

        self.relu = layers.ReLU()

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)

        if self.use_spatial_attention:
            x = self.spatial_attention(x)
        if self.use_channel_attention:
            x = self.channel_attention(x)

        if inputs.shape[-1] == self.filters:
            x += inputs

        return self.relu(x)

# -----------------------------
# Final Model (Case 7)
# -----------------------------
def build_cnn_case7_model(image_width=224, image_height=224, num_classes=2):
    inputs = layers.Input(shape=(image_width, image_height, 3))
    x = layers.Rescaling(1.0 / 255)(inputs)

    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    # Initial conv layers
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # Residual Blocks — 16 layers total, NO attention
    x = ResidualAttentionBlock(128, use_spatial_attention=False, use_channel_attention=False)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_spatial_attention=False, use_channel_attention=False)(x)

    x = ResidualAttentionBlock(256, use_spatial_attention=False, use_channel_attention=False)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_spatial_attention=False, use_channel_attention=False)(x)

    x = ResidualAttentionBlock(512, use_spatial_attention=False, use_channel_attention=False)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(512, use_spatial_attention=False, use_channel_attention=False)(x)

    # Self-Attention not used in Case 7

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)

    # Dropout = 0.3 for Case 7
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    if num_classes == 2:
        outputs = layers.Dense(1, activation='sigmoid')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(num_classes, activation='softmax')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()

    model = Model(inputs, outputs, name="CNN_Case7")

    model.compile(
        optimizer=optimizers.Adam(1e-4),
        loss=loss_fn,
        metrics=[
            'accuracy',
            metrics.Precision(name="precision"),
            metrics.Recall(name="recall"),
            metrics.AUC(name="auc")
        ]
    )

    return model

# -----------------------------
# Testing the model
# -----------------------------
if __name__ == "__main__":
    model = build_cnn_case7_model()
    model.summary()
    dummy_input = tf.random.normal((1, 224, 224, 3))
    output = model(dummy_input)
    print("Output shape:", output.shape)


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=200
)

# Case 8: Data Augmentation using Enhanced CGAN,	Dropout: 0.3	No. of epochs: 100, No. of layers: 20	Self-attention: YES,	Spatial attention: YES

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


# Load the Training and Validation Dataset
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'training',
    seed = 0
)

validation_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'validation',
    seed = 0
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


# Classification

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses
import numpy as np
from tensorflow.keras import metrics

# ================================
# Self-Attention Layer
# ================================
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"

        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        input_shape = tf.shape(inputs)
        height, width = input_shape[1], input_shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, input_shape[3]))

        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)

        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention_output, _ = self.attention(query, key, value)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        output = self.combine_heads(concat_attention)
        output = tf.reshape(output, (batch_size, height, width, self.embed_dim))
        return output

# ================================
# Spatial Attention Block
# ================================
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)
        self.softmax = layers.Softmax(axis=-1)

    def call(self, inputs):
        batch_size, height, width, _ = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]

        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)

        query = tf.reshape(query, (batch_size, height * width, self.filters))
        key = tf.reshape(key, (batch_size, height * width, self.filters))
        value = tf.reshape(value, (batch_size, height * width, self.filters))

        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = attention_scores / tf.math.sqrt(tf.cast(self.filters, tf.float32))
        attention_weights = self.softmax(attention_scores)

        attended_features = tf.matmul(attention_weights, value)
        attended_features = tf.reshape(attended_features, (batch_size, height, width, self.filters))
        output = self.conv_out(attended_features)

        if inputs.shape[-1] == self.filters:
            output = output + inputs
        return output

# ================================
# Channel Attention Block
# ================================
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio

        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')

    def call(self, inputs):
        avg_pool = self.global_avg_pool(inputs)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)
        avg_pool = self.dense2(self.dense1(avg_pool))

        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense2(self.dense1(max_pool))

        attention = avg_pool + max_pool
        return inputs * attention

# ================================
# Residual Block with Optional Attention
# ================================
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_spatial=False, **kwargs):
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_spatial = use_spatial

        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()

        if use_spatial:
            self.spatial_attention = SpatialAttentionBlock(filters)  # Enabled Spatial Attention
        self.channel_attention = ChannelAttentionBlock(filters)
        self.relu = layers.ReLU()

    def call(self, inputs):
        x = self.bn1(self.conv1(inputs))
        x = self.bn2(self.conv2(x))

        if self.use_spatial:
            x = self.spatial_attention(x)  # Apply Spatial Attention if enabled
        x = self.channel_attention(x)      # Always apply Channel Attention

        if inputs.shape[-1] == self.filters:
            x = x + inputs
        return self.relu(x)

# ================================
# Main CNN Model Definition
# ================================
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    x = layers.Rescaling(1./255)(inputs)

    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # 20 total conv/attention/residual layers including these blocks
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = ResidualAttentionBlock(128, use_spatial=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_spatial=True)(x)
    x = ResidualAttentionBlock(128, use_spatial=True)(x)

    x = ResidualAttentionBlock(256, use_spatial=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_spatial=True)(x)
    x = ResidualAttentionBlock(256, use_spatial=True)(x)

    x = ResidualAttentionBlock(512, use_spatial=True)(x)
    x = layers.MaxPooling2D(2)(x)

    x = SelfAttention(embed_dim=512, num_heads=8)(x)  # Self-attention enabled
    x = ResidualAttentionBlock(512, use_spatial=True)(x)
    x = ResidualAttentionBlock(512, use_spatial=True)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dropout(0.3)(x)  # Dropout set to 0.3
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()

    model = Model(inputs, outputs, name="CNN_SelfAttention")
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss=loss_fn,
        metrics=[
            'accuracy',
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc')
        ]
    )
    return model

# Usage example
if __name__ == "__main__":
    IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES = 224, 224, 2
    print("Building CNN with Self-Attention and Spatial Attention (Case 8)...")
    model = build_cnn_attention_model(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES)
    model.summary()
    dummy_input = tf.random.normal((1, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    output = model(dummy_input)
    print(f"\nModel output shape: {output.shape}")
    print(f"Output value: {output.numpy()}")


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=100
)

# Case 9: Data Augmentation using Enhanced CGAN,	Dropout: 0.7	No. of epochs: 100, No. of layers: 20	Self-attention: YES,	Spatial attention: YES

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


# Load the Training and Validation Dataset
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'training',
    seed = 0
)

validation_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'validation',
    seed = 0
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


# Classification

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses
import numpy as np
from tensorflow.keras import metrics

# ================================
# Self-Attention Layer
# ================================
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"

        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        input_shape = tf.shape(inputs)
        height, width = input_shape[1], input_shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, input_shape[3]))

        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)

        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention_output, _ = self.attention(query, key, value)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        output = self.combine_heads(concat_attention)
        output = tf.reshape(output, (batch_size, height, width, self.embed_dim))
        return output

# ================================
# Spatial Attention Block
# ================================
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)
        self.softmax = layers.Softmax(axis=-1)

    def call(self, inputs):
        batch_size, height, width, _ = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]

        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)

        query = tf.reshape(query, (batch_size, height * width, self.filters))
        key = tf.reshape(key, (batch_size, height * width, self.filters))
        value = tf.reshape(value, (batch_size, height * width, self.filters))

        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = attention_scores / tf.math.sqrt(tf.cast(self.filters, tf.float32))
        attention_weights = self.softmax(attention_scores)

        attended_features = tf.matmul(attention_weights, value)
        attended_features = tf.reshape(attended_features, (batch_size, height, width, self.filters))
        output = self.conv_out(attended_features)

        if inputs.shape[-1] == self.filters:
            output = output + inputs
        return output

# ================================
# Channel Attention Block
# ================================
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio

        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')

    def call(self, inputs):
        avg_pool = self.global_avg_pool(inputs)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)
        avg_pool = self.dense2(self.dense1(avg_pool))

        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense2(self.dense1(max_pool))

        attention = avg_pool + max_pool
        return inputs * attention

# ================================
# Residual Block
# ================================
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_spatial=False, **kwargs):
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_spatial = use_spatial

        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()

        if use_spatial:
            self.spatial_attention = SpatialAttentionBlock(filters)  # Enabled Spatial Attention
        self.channel_attention = ChannelAttentionBlock(filters)
        self.relu = layers.ReLU()

    def call(self, inputs):
        x = self.bn1(self.conv1(inputs))
        x = self.bn2(self.conv2(x))

        if self.use_spatial:
            x = self.spatial_attention(x)  # Apply Spatial Attention if enabled
        x = self.channel_attention(x)      # Always apply Channel Attention

        if inputs.shape[-1] == self.filters:
            x = x + inputs
        return self.relu(x)

# ================================
# Main CNN Model Definition
# ================================
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    x = layers.Rescaling(1./255)(inputs)

    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # 20 total conv/attention/residual layers including these blocks
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = ResidualAttentionBlock(128, use_spatial=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_spatial=True)(x)
    x = ResidualAttentionBlock(128, use_spatial=True)(x)

    x = ResidualAttentionBlock(256, use_spatial=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_spatial=True)(x)
    x = ResidualAttentionBlock(256, use_spatial=True)(x)

    x = ResidualAttentionBlock(512, use_spatial=True)(x)
    x = layers.MaxPooling2D(2)(x)

    x = SelfAttention(embed_dim=512, num_heads=8)(x)  # Self-attention enabled
    x = ResidualAttentionBlock(512, use_spatial=True)(x)
    x = ResidualAttentionBlock(512, use_spatial=True)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dropout(0.7)(x)  # Dropout set to 0.3
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.7)(x)

    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()

    model = Model(inputs, outputs, name="CNN_SelfAttention")
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss=loss_fn,
        metrics=[
            'accuracy',
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc')
        ]
    )
    return model

# Usage example
if __name__ == "__main__":
    IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES = 224, 224, 2
    print("Building CNN with Self-Attention and Spatial Attention (Case 9)...")
    model = build_cnn_attention_model(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES)
    model.summary()
    dummy_input = tf.random.normal((1, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    output = model(dummy_input)
    print(f"\nModel output shape: {output.shape}")
    print(f"Output value: {output.numpy()}")


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=100
)

# Case 10: Data Augmentation using Enhanced CGAN,	Dropout: 0.3	No. of epochs: 300, No. of layers: 17	Self-attention: YES,	Spatial attention: YES

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


# Load the Training and Validation Dataset
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'training',
    seed = 0
)

validation_ds = image_dataset_from_directory(
    TRAIN_PATH,
    labels = 'inferred',
    validation_split = 0.3,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    subset = 'validation',
    seed = 0
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


# Classification

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses
import numpy as np
from tensorflow.keras import metrics

# Self-Attention Layer
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim                      # Total embedding dimension
        self.num_heads = num_heads                      # Number of attention heads
        self.head_dim = embed_dim // num_heads          # Dimension per attention head
        
        # Ensure embedding dimension is divisible by number of heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
        
        # Dense layers to project inputs into queries, keys, and values
        self.query_dense = layers.Dense(embed_dim)  
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        
        # Dense layer to combine multiple attention heads output
        self.combine_heads = layers.Dense(embed_dim)
        
    def attention(self, query, key, value):
        # Calculate scaled dot-product attention scores
        score = tf.matmul(query, key, transpose_b=True)   # Compute dot product of query and key^T
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32) # Dimensionality for scaling
        scaled_score = score / tf.math.sqrt(dim_key)      # Scale scores to stabilize gradients
        weights = tf.nn.softmax(scaled_score, axis=-1)    # Apply softmax to get attention weights
        output = tf.matmul(weights, value)                 # Weighted sum of values based on attention weights
        return output, weights
    
    def separate_heads(self, x, batch_size):
        # Reshape the input tensor to split into multiple attention heads
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim)) 
        # Transpose to shape: (batch_size, num_heads, sequence_length, head_dim)
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        
        # Flatten spatial dimensions (height and width) into one sequence dimension
        input_shape = tf.shape(inputs)
        height, width = input_shape[1], input_shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, input_shape[3]))  # shape: (batch, seq_len, channels)
        
        # Create queries, keys, and values projections
        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)
        
        # Separate each into multiple heads
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        
        # Compute attention output and weights
        attention_output, attention_weights = self.attention(query, key, value)
        
        # Transpose back to (batch_size, seq_len, embed_dim)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        
        # Concatenate attention heads
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        
        # Final dense layer to combine heads into output
        output = self.combine_heads(concat_attention)
        
        # Reshape output back to spatial format (batch_size, height, width, embed_dim)
        output = tf.reshape(output, (batch_size, height, width, self.embed_dim))
        
        return output

# Spatial Attention Block
#SpatialAttentionBlock is designed to perform spatial attention—that is, it learns where in the image the model should "pay more attention" by assigning weights to each spatial location
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        # 1x1 convolutions for query, key, and value feature generation
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)  # Final convolution after attention
        self.softmax = layers.Softmax(axis=-1)    # Softmax for attention weights
        
    def call(self, inputs):
        # Get dynamic shapes for batch size, height, width, channels
        batch_size, height, width, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        
        # Generate query, key, and value feature maps from input
        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)
        
        # Reshape to (batch_size, sequence_length, filters) for matrix multiplication
        query = tf.reshape(query, (batch_size, height * width, self.filters))
        key = tf.reshape(key, (batch_size, height * width, self.filters))
        value = tf.reshape(value, (batch_size, height * width, self.filters))
        
        # Compute attention scores by dot product of query and key transpose
        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = attention_scores / tf.math.sqrt(tf.cast(self.filters, tf.float32))  # Scale scores
        
        # Softmax to obtain attention weights
        attention_weights = self.softmax(attention_scores)
        
        # Weighted sum of values based on attention weights
        attended_features = tf.matmul(attention_weights, value)
        
        # Reshape attended features back to spatial format
        attended_features = tf.reshape(attended_features, (batch_size, height, width, self.filters))
        
        # Final convolution to refine features
        output = self.conv_out(attended_features)
        
        # Add residual connection if input channels match filter count
        if inputs.shape[-1] == self.filters:
            output = output + inputs
        
        return output

# Channel Attention Block
#In Channel Attention, the model learns to weigh the importance of each channel in a feature map.
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio
        
        # Global average and max pooling layers
        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        
        # Dense layers to learn channel-wise attention weights
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')
        
    def call(self, inputs):
        # Average pooling branch: captures average channel information
        avg_pool = self.global_avg_pool(inputs)                     # Shape: (batch_size, channels)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)   # Reshape to (batch_size, 1, 1, channels)
        avg_pool = self.dense1(avg_pool)                             # Reduce dimension and non-linearity
        avg_pool = self.dense2(avg_pool)                             # Channel attention weights via sigmoid
        
        # Max pooling branch: captures prominent channel features
        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense1(max_pool)
        max_pool = self.dense2(max_pool)
        
        # Combine attention weights from both branches
        attention = avg_pool + max_pool
        
        # Multiply input features by attention weights (channel-wise scaling)
        return inputs * attention

# ResNet-like Block with Attention
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_attention=True, **kwargs):
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_attention = use_attention
        
        # Two convolutional layers with batch normalization
        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()
        
        # Attention blocks (spatial and channel) if enabled
        if use_attention:
            self.spatial_attention = SpatialAttentionBlock(filters)
            self.channel_attention = ChannelAttentionBlock(filters)
        
        self.relu = layers.ReLU()
        
    def call(self, inputs):
        # First conv layer with activation and batch norm
        x = self.conv1(inputs)
        x = self.bn1(x)
        
        # Second conv layer with batch norm (no activation yet)
        x = self.conv2(x)
        x = self.bn2(x)
        
        # Apply attention blocks if enabled
        if self.use_attention:
            x = self.spatial_attention(x)
            x = self.channel_attention(x)
        
        # Residual skip connection if input and output have same channel dimension
        if inputs.shape[-1] == self.filters:
            x = x + inputs
        
        # Final activation after residual addition
        return self.relu(x)

# CNN Model with Self-Attention
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)  # 1 conv
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Stage 1: 2 conv layers
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)  # 1 conv
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)  # 1 conv
    x = layers.BatchNormalization()(x)
    
    # Stage 2: 2 ResidualAttentionBlocks (4 conv layers)
    x = ResidualAttentionBlock(128, use_attention=True)(x)  # 2 conv layers
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)  # 2 conv layers
    
    # Stage 3: 2 ResidualAttentionBlocks (4 conv layers)
    x = ResidualAttentionBlock(256, use_attention=True)(x)  # 2 conv layers
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)  # 2 conv layers
    
    # Stage 4: 3 ResidualAttentionBlocks (6 conv layers)
    x = ResidualAttentionBlock(512, use_attention=True)(x)  # 2 conv layers
    x = layers.MaxPooling2D(2)(x)
    
    x = SelfAttention(embed_dim=512, num_heads=8)(x)  # no conv layers
    
    x = ResidualAttentionBlock(512, use_attention=True)(x)  # 2 conv layers
    x = ResidualAttentionBlock(512, use_attention=True)(x)  # 2 conv layers
    x = ResidualAttentionBlock(512, use_attention=True)(x)  # 2 conv layers
    
    # Global avg pooling and classifier head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()
    
    model = Model(inputs, outputs, name="CNN_SelfAttention_case10")
    optimizer = optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=[
            'accuracy',
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc')
        ]
    )
    return model


# Usage example
if __name__ == "__main__":
    # Define input image dimensions and number of classes
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    NUM_CLASSES = 2
    
    print("Building CNN with Self-Attention model for Case10...")
    
    # Build the model
    model = build_cnn_attention_model(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES)
    
    # Print the model summary showing layers and parameters
    model.summary()
    
    # Generate a dummy input tensor (batch size 1, 224x224 RGB image)
    dummy_input = tf.random.normal((1, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    
    # Run the model on dummy data to test forward pass
    output = model(dummy_input)
    
    # Print output tensor shape
    print(f"\nModel output shape: {output.shape}")
    
    # Print actual output values (probabilities or logits)
    print(f"Output value: {output.numpy()}")


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=300
)